# 중고차 가격 예측하기

<span style="color:blue"> 환경 준비 </span>

In [8]:
# 라이브러리 불러오기
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

In [51]:
# 데이터 읽어오기
path = 'D:\PRACTICE\\used_cars.csv'
data = pd.read_csv(path)

<br/>
<span style="color:blue"> 데이터 이해 </span>

In [10]:
# 상위 몇 개 행 확인
data.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


In [29]:
data.tail()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
4004,Bentley,Continental GT Speed,2023,714 mi.,Gasoline,6.0L W12 48V PDI DOHC Twin Turbo,8-Speed Automatic with Auto-Shift,C / C,Hotspur,None reported,Yes,"$349,950"
4005,Audi,S4 3.0T Premium Plus,2022,"10,900 mi.",Gasoline,349.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,"$53,900"
4006,Porsche,Taycan,2022,"2,116 mi.",NaN,Electric,Automatic,Black,Black,None reported,NaN,"$90,998"
4007,Ford,F-150 Raptor,2020,"33,000 mi.",Gasoline,450.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,Blue,Black,None reported,Yes,"$62,999"
4008,BMW,X3 xDrive30i,2020,"43,000 mi.",Gasoline,248.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Brown,At least 1 accident or damage reported,Yes,"$40,000"


In [14]:
# 변수 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


* **Brand & Model:** Identify the brand or company name along with the specific model of each vehicle.
* **Model Year:** Discover the manufacturing year of the vehicles, crucial for assessing depreciation and technology advancements.
* **Mileage:** Obtain the mileage of each vehicle, a key indicator of wear and tear and potential maintenance requirements.
* **Fuel Type:** Learn about the type of fuel the vehicles run on, whether it's gasoline, diesel, electric, or hybrid.
* **Engine Type:** Understand the engine specifications, shedding light on performance and efficiency.
* **Transmission:** Determine the transmission type, whether automatic, manual, or another variant.
* **Exterior & Interior Colors:** Explore the aesthetic aspects of the vehicles, including exterior and interior color options.
* **Accident History:** Discover whether a vehicle has a prior history of accidents or damage, crucial for informed decision-making.
* **Clean Title:** Evaluate the availability of a clean title, which can impact the vehicle's resale value and legal status.
* **Price:** Access the listed prices for each vehicle, aiding in price comparison and budgeting

<br/>
<span style="color:blue"> 데이터 준비 </span>

In [16]:
# 결측치 확인
data.isna().sum()

brand             0
model             0
model_year        0
milage            0
fuel_type       170
engine            0
transmission      0
ext_col           0
int_col           0
accident        113
clean_title     596
price             0
dtype: int64

In [17]:
# fuel_type 결측치 처리
# model 값이 동일한 행의 fuel_type 값을 넣어줌

In [53]:
# 모델을 기준으로 정렬
data = data.sort_values('model')
data

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
704,FIAT,124 Spider Abarth,2017,"45,000 mi.",Gasoline,164.0HP 1.4L 4 Cylinder Engine Gasoline Fuel,6-Speed M/T,Gray,Black,At least 1 accident or damage reported,Yes,"$22,500"
2900,BMW,128 i,2013,"67,874 mi.",Gasoline,230.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed A/T,Gray,Black,None reported,Yes,"$18,000"
509,BMW,135 i,2008,"87,000 mi.",Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,A/T,White,Beige,None reported,Yes,"$15,300"
1507,BMW,135 i,2011,"132,000 mi.",Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,7-Speed A/T,Black,Beige,None reported,Yes,"$11,000"
715,BMW,135 i,2009,"72,900 mi.",Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed A/T,Gray,Gray,None reported,Yes,"$15,500"
...,...,...,...,...,...,...,...,...,...,...,...,...
3855,Scion,tC Anniversary Edition,2014,"99,999 mi.",Gasoline,179.0HP 2.5L 4 Cylinder Engine Gasoline Fuel,6-Speed M/T,White,Black,None reported,Yes,"$10,998"
1284,Scion,tC Anniversary Edition,2014,"115,000 mi.",Gasoline,179.0HP 2.5L 4 Cylinder Engine Gasoline Fuel,6-Speed M/T,Silver,Silver,None reported,Yes,"$11,495"
177,Scion,tC Base,2013,"177,600 mi.",Gasoline,180.0HP 2.5L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,Silver,Black,At least 1 accident or damage reported,Yes,"$7,100"
690,Scion,tC Release Series 6.0,2010,"120,010 mi.",Gasoline,161.0HP 2.4L 4 Cylinder Engine Gasoline Fuel,4-Speed A/T,Gray,Black,At least 1 accident or damage reported,Yes,"$6,500"


In [95]:
df = data.groupby('model', as_index = False)[['fuel_type']].count()
df

,model,fuel_type
0,124 Spider Abarth,1
1,128 i,1
2,135 i,3
3,135 is,1
4,1500 Big Horn,11
...,...,...
1893,i8 Base,5
1894,tC Anniversary Edition,2
1895,tC Base,1
1896,tC Release Series 6.0,1


In [66]:
# fuel_type 값이 NaN인데 동일한 model 값도 존재하지 않는 경우
df[df['fuel_type'] == 0]

,model,fuel_type
18,2 Launch Edition,0
95,500e Battery Electric,0
258,Air Grand Touring,0
259,Air Pure,0
309,Bolt EUV Premier,0
...,...,...
1887,e-tron Premium,0
1888,e-tron Prestige,0
1890,i3 94 Ah,0
1891,i3 Base,0


In [96]:
type(df)

pandas.core.frame.DataFrame

In [76]:
# fuel_type 열의 최빈값
data['fuel_type'].mode()

0    Gasoline
dtype: object

In [143]:
for index, row in data.iterrows() :
    if (type(row.fuel_type) == str) and (len(row.fuel_type) >= 2) :
        continue
        
    else :  
        d = df.loc[df['model'] == row.model]
        
        if (d['fuel_type'].any() != 0) :
            try : 
                data.fuel_type.fillna(method = 'bfill', limit = 1, inplace = True)
            except :
                continue
                
        else :
            try :
                data.fuel_type.fillna('Gasoline', inplace = True)
            except :
                continue

data.isna().sum()

brand             0
model             0
model_year        0
milage            0
fuel_type         0
engine            0
transmission      0
ext_col           0
int_col           0
accident        113
clean_title     596
price             0
dtype: int64

In [144]:
data = data.sort_values('fuel_type')
data

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
1940,Ford,F-250 Lariat,2019,"85,000 mi.",Diesel,450.0HP 6.7L 8 Cylinder Engine Diesel Fuel,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$52,500"
1059,Chevrolet,Silverado 3500 High Country,2015,"77,500 mi.",Diesel,397.0HP 6.6L 8 Cylinder Engine Diesel Fuel,6-Speed A/T,Black,Brown,None reported,Yes,"$54,000"
1281,Chevrolet,Silverado 3500 LTZ,2022,"85,200 mi.",Diesel,445.0HP 6.6L 8 Cylinder Engine Diesel Fuel,10-Speed A/T,Silver,Gray,None reported,Yes,"$67,500"
1635,Chevrolet,Silverado 3500 LTZ,2018,"92,149 mi.",Diesel,6.6L V8 32V DDI OHV Turbo Diesel,6-Speed Automatic,Black,Jet Black,NaN,NaN,"$52,889"
502,Chevrolet,Express 3500 LT,2016,"120,000 mi.",Diesel,260.0HP 6.6L 8 Cylinder Engine Diesel Fuel,6-Speed A/T,White,Gray,None reported,Yes,"$19,500"
...,...,...,...,...,...,...,...,...,...,...,...,...
2303,Acura,NSX Base,1993,"75,980 mi.",–,–,A/T,Silver,Black,None reported,Yes,"$90,200"
2103,Volvo,850 Turbo,1995,"94,000 mi.",–,–,A/T,White,Black,None reported,Yes,"$4,500"
855,Ford,Bronco,1974,"6,217 mi.",–,–,–,Dark Gray Metallic,–,None reported,Yes,"$115,000"
3213,Ford,Mustang EcoBoost Premium,2019,"31,000 mi.",–,–,6-Speed M/T,Gray,Black,None reported,Yes,"$34,700"


In [140]:
data['fuel_type'].unique()

array(['Gasoline', 'Diesel', 'Hybrid', '–', 'E85 Flex Fuel', nan,
       'Plug-In Hybrid', 'not supported'], dtype=object)

In [145]:
data = data.sort_values('model')
data

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
704,FIAT,124 Spider Abarth,2017,"45,000 mi.",Gasoline,164.0HP 1.4L 4 Cylinder Engine Gasoline Fuel,6-Speed M/T,Gray,Black,At least 1 accident or damage reported,Yes,"$22,500"
2900,BMW,128 i,2013,"67,874 mi.",Gasoline,230.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed A/T,Gray,Black,None reported,Yes,"$18,000"
715,BMW,135 i,2009,"72,900 mi.",Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed A/T,Gray,Gray,None reported,Yes,"$15,500"
1507,BMW,135 i,2011,"132,000 mi.",Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,7-Speed A/T,Black,Beige,None reported,Yes,"$11,000"
509,BMW,135 i,2008,"87,000 mi.",Gasoline,300.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,A/T,White,Beige,None reported,Yes,"$15,300"
...,...,...,...,...,...,...,...,...,...,...,...,...
3855,Scion,tC Anniversary Edition,2014,"99,999 mi.",Gasoline,179.0HP 2.5L 4 Cylinder Engine Gasoline Fuel,6-Speed M/T,White,Black,None reported,Yes,"$10,998"
1284,Scion,tC Anniversary Edition,2014,"115,000 mi.",Gasoline,179.0HP 2.5L 4 Cylinder Engine Gasoline Fuel,6-Speed M/T,Silver,Silver,None reported,Yes,"$11,495"
177,Scion,tC Base,2013,"177,600 mi.",Gasoline,180.0HP 2.5L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,Silver,Black,At least 1 accident or damage reported,Yes,"$7,100"
690,Scion,tC Release Series 6.0,2010,"120,010 mi.",Gasoline,161.0HP 2.4L 4 Cylinder Engine Gasoline Fuel,4-Speed A/T,Gray,Black,At least 1 accident or damage reported,Yes,"$6,500"


In [146]:
# '–'로 잘못 표기된 NaN 값들을 NaN으로 바꿔주기
for index, row in data.iterrows() :
    if (type(row.fuel_type) == str) and (len(row.fuel_type) >= 2) :
        continue
        
    else :  
        data.loc[index, 'fuel_type'] = np.NaN

In [147]:
data.isna().sum()

brand             0
model             0
model_year        0
milage            0
fuel_type        45
engine            0
transmission      0
ext_col           0
int_col           0
accident        113
clean_title     596
price             0
dtype: int64

In [148]:
# NaN 값들을 다시 한 번 bfill로 채워줌
for index, row in data.iterrows() :
    if (type(row.fuel_type) == str) and (len(row.fuel_type) >= 2) :
        continue
        
    else :
        try : 
            data.fuel_type.fillna(method = 'bfill', limit = 1, inplace = True)
        except :
            continue

In [149]:
data = data.sort_values('fuel_type')
data

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
3292,Mercedes-Benz,Sprinter 2500 Standard Roof,2021,"13,344 mi.",Diesel,188.0HP 3.0L V6 Cylinder Engine Diesel Fuel,7-Speed A/T,Red,Black,None reported,Yes,"$56,500"
3550,Ford,F-350 King Ranch,2015,"92,421 mi.",Diesel,440.0HP 6.7L 8 Cylinder Engine Diesel Fuel,6-Speed A/T,White,Brown,None reported,Yes,"$51,900"
3708,Ford,F-350 Lariat,2018,"66,281 mi.",Diesel,450.0HP 6.7L 8 Cylinder Engine Diesel Fuel,Transmission w/Dual Shift Mode,Gray,Black,None reported,Yes,"$63,500"
3261,Ford,F-350 Lariat,2015,"162,000 mi.",Diesel,440.0HP 6.7L 8 Cylinder Engine Diesel Fuel,Transmission w/Dual Shift Mode,Red,Beige,None reported,Yes,"$32,000"
2170,Ford,F-350 Lariat,2021,"28,220 mi.",Diesel,475.0HP 6.7L 8 Cylinder Engine Diesel Fuel,10-Speed A/T,Silver,Black,None reported,Yes,"$73,600"
...,...,...,...,...,...,...,...,...,...,...,...,...
3825,Toyota,Prius Plug-in Base,2014,"106,000 mi.",Plug-In Hybrid,134.0HP 1.8L 4 Cylinder Engine Plug-In Electri...,A/T,Green,Beige,At least 1 accident or damage reported,Yes,"$11,800"
1820,Volvo,XC90 Recharge Plug-In Hybrid T8 Inscription 7 ...,2022,"7,800 mi.",Plug-In Hybrid,455.0HP 2.0L 4 Cylinder Engine Plug-In Electri...,8-Speed A/T,Black,Beige,None reported,Yes,"$66,000"
1509,Hyundai,IONIQ 5 SE,2022,"18,500 mi.",Plug-In Hybrid,320.0HP Electric Motor Electric Fuel System,A/T,White,Black,None reported,Yes,"$42,000"
3700,Toyota,Mirai Base,2016,"40,000 mi.",not supported,151.0HP Electric Motor Hydrogen Fuel,A/T,Silver,Black,None reported,Yes,"$9,500"


In [150]:
data['fuel_type'].unique()

array(['Diesel', 'E85 Flex Fuel', 'Gasoline', 'Hybrid', 'Plug-In Hybrid',
       'not supported'], dtype=object)